# [おためし] 百言語翻訳機 

## Facebook の M2M-100 を動かしてみる 

[対応言語一覧はこちら.](https://huggingface.co/facebook/m2m100-12B-last-ckpt#languages-covered)  
本当にぴったり100言語.  
少なくとも日常で聞いたことがある言語は含まれてると思ってよさそう.  
逆に知らない言語の方が多い.  

ただし**1.2B (12億パラメタ; 約5GB)** まで試したところ, 精度は悪い.  
  ((パクチーが別の言葉に訳されたり, 円安と円高がごっちゃになったりする))  
本命の**12B (120億パラメタ; 約47GB)** はローカルの容量が足りず, 動かせていない.  
今後, Google ColaboratoryやSagemaker Studio等で動かせないか調べて試してみたい. 


 (23/06/18) 



## 実行環境 

In [1]:
!conda list '^(python|pytorch|sentencepiece|transformers)$' | tail -n 5


# Name                    Version                   Build  Channel
python                    3.11.3               hf27a42d_1  
pytorch                   1.13.1          cpu_py311h9e40b02_0  
sentencepiece             0.1.99                   pypi_0    pypi
transformers              4.30.2             pyhd8ed1ab_1    conda-forge


## モデル名を指定 

コメントアウトを切り替えて, 翻訳に使用するモデルを選択してください. 


In [2]:
# モデル名 
# model_name = 'facebook/m2m100_418M'  # M2M-100 (4億1800万パラメタ; 約2GB)
model_name = 'facebook/m2m100_1.2B'  # M2M-100 (12億パラメタ; 約5GB)
# model_name = 'facebook/m2m100-12B-last-ckpt'  # M2M-100 (120億パラメタ; 約47GB)


## 1. 諸設定 


In [ ]:
import os

import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer


In [4]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'  # to ignore "OMP: Error #15"


In [5]:
# モデルパラメタとトークナイザをあらかじめ置いておくパス
model_path = os.path.join('models', model_name)  # 例: 'models/facebook/m2m100_418M'

model_path


'models/facebook/m2m100_1.2B'

In [ ]:
%%time

# モデルパラメタを読込
model = M2M100ForConditionalGeneration.from_pretrained(model_path)

model


In [7]:
# トークナイザを読込
tokenizer = M2M100Tokenizer.from_pretrained(model_path)

tokenizer


M2M100Tokenizer(name_or_path='models/facebook/m2m100_1.2B', vocab_size=128104, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'additional_special_tokens': ['__af__', '__am__', '__ar__', '__ast__', '__az__', '__ba__', '__be__', '__bg__', '__bn__', '__br__', '__bs__', '__ca__', '__ceb__', '__cs__', '__cy__', '__da__', '__de__', '__el__', '__en__', '__es__', '__et__', '__fa__', '__ff__', '__fi__', '__fr__', '__fy__', '__ga__', '__gd__', '__gl__', '__gu__', '__ha__', '__he__', '__hi__', '__hr__', '__ht__', '__hu__', '__hy__', '__id__', '__ig__', '__ilo__', '__is__', '__it__', '__ja__', '__jv__', '__ka__', '__kk__', '__km__', '__kn__', '__ko__', '__lb__', '__lg__', '__ln__', '__lo__', '__lt__', '__lv__', '__mg__', '__mk__', '__ml__', '__mn__', '__mr__', '__ms__', '__my__', '__ne__', '__nl__', '__no__', '__n

In [8]:
def translate(input_text, src_lang, target_lang):
    """
    翻訳実行
    
    Args:
        input_text (str): 入力文字列
        src_lang (src): 翻訳「元」言語
        target_lang (src): 翻訳「先」言語
    """
    
    tokenizer.src_lang = src_lang  # 翻訳「元」言語
    
    encoded_inputs = tokenizer(input_text, return_tensors='pt')  # 符号化
    
    outputs = model.generate(
        **encoded_inputs, 
        forced_bos_token_id=tokenizer.get_lang_id(target_lang),  # 翻訳「先」言語
    )
    
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)  # 復号


## 2. 翻訳実行 


### ヒンディー語 → フランス語 


In [9]:
input_text = 'जीवन एक चॉकलेट बॉक्स की तरह है।'

print(input_text)


जीवन एक चॉकलेट बॉक्स की तरह है।


In [ ]:
%%time

output_texts = translate(
    input_text, 
    'hi',  # ヒンディー語
    'fr',  # フランス語
)

print(output_texts[0])


### ヒンディー語 → 日本語 


In [12]:
input_text = 'जीवन एक चॉकलेट बॉक्स की तरह है।'

print(input_text)


जीवन एक चॉकलेट बॉक्स की तरह है।


In [13]:
output_texts = translate(
    input_text, 
    'hi',  # ヒンディー語
    'ja',  # 日本語
)

print(output_texts[0])


人生はチョコレートの箱のようなものです。


### フランス語 → 日本語 


In [14]:
input_text = 'La vie est comme une boîte de chocolat.'

print(input_text)


La vie est comme une boîte de chocolat.


In [15]:
output_texts = translate(
    input_text, 
    'fr',  # フランス語
    'ja',  # 日本語
)

print(output_texts[0])


人生はチョコレートの箱のようなものです。


### 日本語 → 中国語 


In [16]:
input_text = 'パクチーが好きな人もいれば、そうでない人もいる。'

print(input_text)


パクチーが好きな人もいれば、そうでない人もいる。


In [17]:
output_texts = translate(
    input_text, 
    'ja',  # 日本語
    'zh',  # 中国語
)

print(output_texts[0])


有些人喜欢巴菲特,有些人不喜欢。


### 日本語 → 英語 


In [18]:
input_text = 'パクチーが好きな人もいれば、そうでない人もいる。'

print(input_text)


パクチーが好きな人もいれば、そうでない人もいる。


In [19]:
output_texts = translate(
    input_text, 
    'ja',  # 日本語
    'en',  # 英語
)

print(output_texts[0])


Some people like pudding, others don’t.


In [26]:
input_text = '外国為替市場で一方的な円安が続いている。対ユーロでは15年ぶりの安値をつけたほか、対スイスフランでは最安値圏にある。インフレ高止まりで世界の中央銀行が追加的な金融引き締めを模索するなか、日銀の緩和継続姿勢が円売りを促す。政府・日銀がドル売り・円買いに動いた水準に近づき、市場では警戒感が高まってきた。'

print(input_text)


外国為替市場で一方的な円安が続いている。対ユーロでは15年ぶりの安値をつけたほか、対スイスフランでは最安値圏にある。インフレ高止まりで世界の中央銀行が追加的な金融引き締めを模索するなか、日銀の緩和継続姿勢が円売りを促す。政府・日銀がドル売り・円買いに動いた水準に近づき、市場では警戒感が高まってきた。


In [27]:
output_texts = translate(
    input_text, 
    'ja',  # 日本語
    'en',  # 英語
)

print(output_texts[0])


On the foreign exchange market, the one-sided yen continues to rise.The euro has reached its lowest level in 15 years, while the Swiss franc is at its lowest.Inflation has stopped and the world’s central banks are seeking additional financial constraints, while the Japanese central bank’s continued easing stance has prompted the sale of the yen.The government and the Japanese central bank have moved closer to the level of dollar sales and yen purchases, and the markets have become more alert.


### 英語 → 日本語 


In [28]:
input_text = 'On the foreign exchange market, the one-sided yen continues to rise.The euro has reached its lowest level in 15 years, while the Swiss franc is at its lowest.Inflation has stopped and the world’s central banks are seeking additional financial constraints, while the Japanese central bank’s continued easing stance has prompted the sale of the yen.The government and the Japanese central bank have moved closer to the level of dollar sales and yen purchases, and the markets have become more alert.'

print(input_text)


On the foreign exchange market, the one-sided yen continues to rise.The euro has reached its lowest level in 15 years, while the Swiss franc is at its lowest.Inflation has stopped and the world’s central banks are seeking additional financial constraints, while the Japanese central bank’s continued easing stance has prompted the sale of the yen.The government and the Japanese central bank have moved closer to the level of dollar sales and yen purchases, and the markets have become more alert.


In [29]:
output_texts = translate(
    input_text, 
    'en',  # 英語
    'ja',  # 日本語
)

print(output_texts[0])


外国為替市場では、一方的な円は上昇し続けている。ユーロは15年ぶりの低水準に達し、スイスフランは最低水準に達している。インフレは止まっており、世界の中央銀行はさらなる金融制約を求めているが、日本の中央銀行の緩和的な姿勢は円の売却を促した。
